In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]    = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
# import necessary libraries
import matplotlib
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

plt.ion()

from random import randint
import time

from ipywidgets import FloatProgress
from IPython.display import display, clear_output


import numpy as np

import keras
from   keras.datasets   import mnist

from   tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf


In [ ]:
# data load
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# reshape to 1 dim
X_train      = X_train.reshape(np.shape(X_train)[0], 
                               (np.shape(X_train)[1] * np.shape(X_train)[2]))
X_test       = X_test.reshape(np.shape(X_test)[0], 
                              (np.shape(X_test)[1] * np.shape(X_test)[2]))

# change type
X_train      = X_train.astype('float32')
X_test       = X_test.astype('float32')

# normalization
X_train_mean = np.mean(X_train)
X_train_std  = np.std(X_train)
X_train      = (X_train - X_train_mean) / (X_train_std + 1e-10)
X_test       = (X_test  - X_train_mean) / (X_train_std + 1e-10)

# convert class vectors to binary class matrices
class_num    = np.max(y_train) + 1 # consider class 0 
y_train      = keras.utils.to_categorical(y_train, class_num).astype('float32')
y_test       = keras.utils.to_categorical(y_test,  class_num).astype('float32')

print('np.shape(X_train) = (%d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)' % np.shape(y_train))
print('np.shape(X_test)  = (%d, %d)' % np.shape(X_test))
print('np.shape(y_test)  = (%d, %d)' % np.shape(y_test))


In [ ]:
# set parameter for training
initial_learning_rate = 2.0
decay_learning_rate   = 0.999
epoch_num             = 100
mini_batch_size       = (128 * 4)
dropconnect_ratio     = 0.2


In [ ]:
np.random.seed(0)
eps           = 1e-10

sess          = tf.InteractiveSession()
is_training   = tf.placeholder(dtype=tf.bool)

global_step   = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                           global_step=global_step,
                                           decay_steps=10,
                                           decay_rate=decay_learning_rate)


# make model by tensorflow

# input layer
x               = tf.placeholder(tf.float32, [None, 784])

# multiply weight
weight_1        = tf.Variable(tf.random_normal([784, 100], mean=0.0, stddev=0.01))
bias_1          = tf.Variable(tf.random_normal([100],      mean=0.0, stddev=0.01))

# drop connect
mask_weight_1   = tf.Variable(tf.zeros([784, 100]), trainable=False)
droped_weight_1 = weight_1 * mask_weight_1

# 
sum_1           = tf.matmul(x, droped_weight_1) + bias_1

# activation
activated_1     = tf.nn.relu(sum_1)

# multiply weight
weight_2        = tf.Variable(tf.random_normal([100, 10], mean=0.0, stddev=0.01))
bias_2          = tf.Variable(tf.random_normal([10],      mean=0.0, stddev=0.01))

# drop connect
mask_weight_2   = tf.Variable(tf.zeros([100, 10]), trainable=False)
droped_weight_2 = weight_2 * mask_weight_2

# 
sum_2           = tf.matmul(activated_1, droped_weight_2) + bias_2

# output layer
y             = tf.nn.softmax(sum_2)


In [ ]:
# define loss function and optimizer
y_         = tf.placeholder(tf.float32, [None, 10])
loss       = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
optimizer  = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

tf.global_variables_initializer().run()


In [ ]:
# 
train_acc_stock = np.zeros(epoch_num)
test_acc_stock  = np.zeros(epoch_num)
test_acc_max    = 0

fig             = plt.figure(figsize=(12,12),dpi=100)
draw_pitch      = 5

# training
for epoch_i in range(epoch_num):
    
    rand_idx    = np.random.permutation(len(y_train))
    
    for iter_j in range(round((len(rand_idx) / mini_batch_size) - 0.5)):
        
        # set mini batch data
        X_train_mini_batch   = X_train[rand_idx[:mini_batch_size], :]
        y_train_mini_batch   = y_train[rand_idx[:mini_batch_size], :]
        rand_idx             = rand_idx[mini_batch_size:]
        
        # drop connect
        mask_weight_1_tmp = sess.run(mask_weight_1)
        mask_weight_1_tmp = (np.random.uniform(0.0, 1.0, np.shape(mask_weight_1_tmp)) > dropconnect_ratio).astype('float32')
        mask_weight_1.load(mask_weight_1_tmp, sess)

        mask_weight_2_tmp = sess.run(mask_weight_2)
        mask_weight_2_tmp = (np.random.uniform(0.0, 1.0, np.shape(mask_weight_2_tmp)) > dropconnect_ratio).astype('float32')
        mask_weight_2.load(mask_weight_2_tmp, sess)
        
        # learning
        train_step.run({x: X_train_mini_batch, y_: y_train_mini_batch})
        
    # validaiton
    correct_prediction       = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    calc_acc                 = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_acc_stock[epoch_i] = calc_acc.eval({x: X_train, y_: y_train})
    test_acc_stock[epoch_i]  = calc_acc.eval({x: X_test,  y_: y_test})
    
    if (test_acc_max < test_acc_stock[epoch_i]):
        test_acc_max = test_acc_stock[epoch_i]
    
    if (((epoch_i + 1) % draw_pitch) == 0):
        # draw
        ax_1 = fig.add_subplot(1, 3, 1)
        ax_1.cla()
        clear_output(wait=True)
        ax_1.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_1.plot(train_acc_stock[:(epoch_i + 1)])
        ax_1.plot(test_acc_stock[:(epoch_i + 1)])
        plt.ylim(-0.05, 1.05)
        plt.title('accuracy of train data = %.2f%%\naccuracy of test data = %.2f%%\n(best : %.2f%%)' % ((train_acc_stock[epoch_i] * 100), (test_acc_stock[epoch_i] * 100), (test_acc_max * 100)))
        display(fig)

        ax_2 = fig.add_subplot(2, 3, 2)
        ax_2.cla()
        clear_output(wait=True)
        ax_2.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        weight_1_tmp = sess.run(weight_1)
        weight_1_tmp = np.sort(weight_1_tmp.flat[:])
        ax_2.plot(weight_1_tmp)
        plt.title('plot weight of 1st layer\n(sorted by value)')
        display(fig)

        ax_3 = fig.add_subplot(2, 3, 3)
        ax_3.cla()
        clear_output(wait=True)
        ax_3.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_3.hist(weight_1_tmp, bins=50)
        plt.yscale('log')
        plt.title('histogram weight of 1st layer\n(std : %.3f)' % np.std(weight_1_tmp))
        display(fig)

        ax_4 = fig.add_subplot(2, 3, 5)
        ax_4.cla()
        clear_output(wait=True)
        ax_4.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        weight_2_tmp = sess.run(weight_2)
        weight_2_tmp = np.sort(weight_2_tmp.flat[:])
        ax_4.plot(weight_2_tmp)
        plt.title('plot weight of 2nd layer\n(sorted by value)')
        display(fig)

        ax_5 = fig.add_subplot(2, 3, 6)
        ax_5.cla()
        clear_output(wait=True)
        ax_5.grid(which='major',color=[0.7, 0.7, 0.7],linestyle='-')
        ax_5.hist(weight_2_tmp, bins=50)
        plt.yscale('log')
        plt.title('histogram weight of 2nd layer\n(std : %.3f)' % np.std(weight_2_tmp))
        display(fig)
